In [1]:
import pandas as pd
import glob
import json
from tqdm import tqdm
import re
from datetime import datetime
import gensim
import logging
import os
import shutil

/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore

In [ ]:
!pip install pysrt

In [3]:
import pysrt


kaggle datasets download -d goweiting/ted-talks-transcript    
Use the Kaggle API to download dataset files.      
https://github.com/Kaggle/kaggle-api    

https://www.kaggle.com/goweiting/ted-talks-transcript/data

Doc2vec   
https://medium.com/scaleabout/a-gentle-introduction-to-doc2vec-db3e8c0cce5e

In [ ]:
!ls -lah ~/.kaggle/datasets/goweiting/ted-talks-transcript/

In [ ]:
#!unzip /Users/nidhin/.kaggle/datasets/goweiting/ted-talks-transcript/tedDirector.zip

In [4]:
base_path= os.path.expanduser("~/.kaggle/datasets/goweiting/ted-talks-transcript/")

In [ ]:
#shutil.rmtree(base_path, ignore_errors=True)
#os.makedirs(base_path)

In [ ]:
!ls /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript


In [ ]:
!kaggle datasets download -d goweiting/ted-talks-transcript -p {base_path}

In [ ]:
!ls /Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript

In [ ]:
!ls {base_path}

In [ ]:
!unzip -o {base_path}/ted-talks-transcript.zip 

In [ ]:
print(f"{base_path}subtitles/*.info.json")

In [5]:
meta_data_files = glob.glob(f"{base_path}subtitles/*/*.info.json")




In [6]:
len(meta_data_files)

2531

In [7]:
def parse_video_info(metadata):
    keys = ["webpage_url", "view_count", "upload_date", "title", "thumbnail" 
            , "duration" , "dislike_count" , "like_count"
            , "description", "categories" ,"id", "average_rating"
            ]
    
    new_dict = {}
    for key in keys:
        new_dict[key] = metadata.get(key,None)
        
    return new_dict

def get_metadata(meta_data_files) -> pd.DataFrame:
    parsed_metadatas = []
    for file in tqdm(meta_data_files):
        with open(file) as f:
            content = json.load(f)
        
        parsed_metadata = parse_video_info(content)
        parsed_metadatas.append(parsed_metadata)

    return pd.DataFrame(parsed_metadatas)


In [8]:
metadata_df = get_metadata(meta_data_files)

100%|██████████| 2531/2531 [00:02<00:00, 1034.90it/s]


In [9]:
metadata_df.head()

average_rating               categories  \
0        4.729469  [Nonprofits & Activism]   
1        4.940791        [News & Politics]   
2        4.554828        [News & Politics]   
3        4.706122   [Science & Technology]   
4        4.732777        [Travel & Events]   

                                         description  dislike_count  duration  \
0  http://www.ted.com When German psychologist In...           14.0       830   
1  As a child growing up in North Korea, Hyeonseo...         2151.0       735   
2  http://www.ted.com Every day, in a city the si...           68.0       938   
3  http://www.ted.com With his team at SENSEable ...           54.0       947   
4  Almost everything we own and use, at some poin...           96.0       684   

            id  like_count                                          thumbnail  \
0  Jc2F3-nawnI       193.0   https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg   
1  PdxPCeWw75k    143165.0  https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...   
2  CLWRclarri0       543.0   https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg   
3  CijsvAGU6-c       681.0   https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg   
4  j7RsRnYlz7I      1341.0  https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...   

                                               title upload_date  view_count  \
0  Inge Missmahl brings peace to the minds of Afg...    20100929       20695   
1          My escape from North Korea | Hyeonseo Lee    20130320     6702386   
2          Carolyn Steel: How food shapes our cities    20091005      125975   
3  Carlo Ratti: Architecture that senses and resp...    20110503       78537   
4   Rose George: Inside the secret shipping industry    20131213      144699   

                                   webpage_url  
0  https://www.youtube.com/watch?v=Jc2F3-nawnI  
1  https://www.youtube.com/watch?v=PdxPCeWw75k  
2  https://www.youtube.com/watch?v=CLWRclarri0  
3  https://www.youtube.com/watch?v=CijsvAGU6-c  
4  https://www.youtube.com/watch?v=j7RsRnYlz7I

In [10]:
subtitle_files = glob.glob(f"{base_path}subtitles/*/*.en.vtt")


In [11]:
subtitle_files[0:2]

['/Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/Jc2F3-nawnI/Jc2F3-nawnI.en.vtt',
 '/Users/nidhin.pattaniyil/.kaggle/datasets/goweiting/ted-talks-transcript/subtitles/PdxPCeWw75k/PdxPCeWw75k.en.vtt']

In [12]:
def parse_subtitle(file_name) -> str:
    subs = pysrt.open(file_name)
    content = ""
    for s in subs:
        content = content + s.text_without_tags + "\n"
    return content

def get_subtitles(subtitle_files) -> pd.DataFrame :
    parsed_subtitles = []
    for file in tqdm(subtitle_files):
        
        res = re.match('.*/(.*).en.vtt',file)
        talk_id  = res.group(1)
        content = parse_subtitle(file)
        
        parsed_subtitles.append ({'id': talk_id , 'transcript': content})
        
    return pd.DataFrame(parsed_subtitles)
        

In [13]:
subtitle_files_df = get_subtitles(subtitle_files)

100%|██████████| 2490/2490 [00:35<00:00, 69.51it/s]


In [14]:
subtitle_files_df.head()

id                                         transcript
0  Jc2F3-nawnI  So I want to tell you a story -- an encouragin...
1  PdxPCeWw75k  Translator: Joseph Geni\nReviewer: Morton Bast...
2  CLWRclarri0  How do you feed a city?\nIt's one of the great...
3  CijsvAGU6-c  Good afternoon, everybody.\nI've got something...
4  j7RsRnYlz7I  A couple of years ago,\nHarvard Business Schoo...

In [15]:
len(metadata_df)

2531

In [16]:
len(subtitle_files_df)

2490

In [17]:
df = pd.merge(metadata_df,subtitle_files_df, on='id')

In [18]:
datetime.strptime('20100929', '%Y%m%d')


datetime.datetime(2010, 9, 29, 0, 0)

In [19]:
df.head()

average_rating               categories  \
0        4.729469  [Nonprofits & Activism]   
1        4.940791        [News & Politics]   
2        4.554828        [News & Politics]   
3        4.706122   [Science & Technology]   
4        4.732777        [Travel & Events]   

                                         description  dislike_count  duration  \
0  http://www.ted.com When German psychologist In...           14.0       830   
1  As a child growing up in North Korea, Hyeonseo...         2151.0       735   
2  http://www.ted.com Every day, in a city the si...           68.0       938   
3  http://www.ted.com With his team at SENSEable ...           54.0       947   
4  Almost everything we own and use, at some poin...           96.0       684   

            id  like_count                                          thumbnail  \
0  Jc2F3-nawnI       193.0   https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg   
1  PdxPCeWw75k    143165.0  https://i.ytimg.com/vi/PdxPCeWw75k/maxresdefau...   
2  CLWRclarri0       543.0   https://i.ytimg.com/vi/CLWRclarri0/hqdefault.jpg   
3  CijsvAGU6-c       681.0   https://i.ytimg.com/vi/CijsvAGU6-c/hqdefault.jpg   
4  j7RsRnYlz7I      1341.0  https://i.ytimg.com/vi/j7RsRnYlz7I/maxresdefau...   

                                               title upload_date  view_count  \
0  Inge Missmahl brings peace to the minds of Afg...    20100929       20695   
1          My escape from North Korea | Hyeonseo Lee    20130320     6702386   
2          Carolyn Steel: How food shapes our cities    20091005      125975   
3  Carlo Ratti: Architecture that senses and resp...    20110503       78537   
4   Rose George: Inside the secret shipping industry    20131213      144699   

                                   webpage_url  \
0  https://www.youtube.com/watch?v=Jc2F3-nawnI   
1  https://www.youtube.com/watch?v=PdxPCeWw75k   
2  https://www.youtube.com/watch?v=CLWRclarri0   
3  https://www.youtube.com/watch?v=CijsvAGU6-c   
4  https://www.youtube.com/watch?v=j7RsRnYlz7I   

                                          transcript  
0  So I want to tell you a story -- an encouragin...  
1  Translator: Joseph Geni\nReviewer: Morton Bast...  
2  How do you feed a city?\nIt's one of the great...  
3  Good afternoon, everybody.\nI've got something...  
4  A couple of years ago,\nHarvard Business Schoo...

In [20]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/nidhin.pattaniyil/anaconda3/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [ ]:
?text.TfidfVectorizer

In [22]:
from sklearn.feature_extraction import text
Text=df['transcript'].tolist()
tfidf=text.TfidfVectorizer(input=Text,stop_words="english", tokenizer=LemmaTokenizer())
matrix=tfidf.fit_transform(Text)
#print(matrix.shape)

In [23]:
### Get Similarity Scores using cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
sim_unigram=cosine_similarity(matrix)

In [24]:
sim_unigram

array([[1.        , 0.42328378, 0.55359966, ..., 0.49388626, 0.53364983,
        0.58179026],
       [0.42328378, 1.        , 0.4507271 , ..., 0.43143527, 0.42768029,
        0.45480349],
       [0.55359966, 0.4507271 , 1.        , ..., 0.55640022, 0.57594978,
        0.61223904],
       ...,
       [0.49388626, 0.43143527, 0.55640022, ..., 1.        , 0.51827279,
        0.5542238 ],
       [0.53364983, 0.42768029, 0.57594978, ..., 0.51827279, 1.        ,
        0.56661015],
       [0.58179026, 0.45480349, 0.61223904, ..., 0.5542238 , 0.56661015,
        1.        ]])

In [25]:
def get_similar_articles(x):
    return "\n".join(df['title'].loc[x.argsort()[-5:-1]])
df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [26]:
df['title'][1]

'My escape from North Korea | Hyeonseo Lee'

In [27]:
print (df['similar_articles_unigram'][1])

How I Stopped the Taliban from Shutting Down My School | Sakena Yacoobi | TED Talks
This is what it's like to go undercover in North Korea | Suki Kim
The family I lost in North Korea. And the family I gained | Joseph Kim
What I learned as a prisoner in North Korea | Euna Lee


# Doc2Vec

In [28]:
df.iloc[0]

average_rating                                                        4.72947
categories                                            [Nonprofits & Activism]
description                 http://www.ted.com When German psychologist In...
dislike_count                                                              14
duration                                                                  830
id                                                                Jc2F3-nawnI
like_count                                                                193
thumbnail                    https://i.ytimg.com/vi/Jc2F3-nawnI/hqdefault.jpg
title                       Inge Missmahl brings peace to the minds of Afg...
upload_date                                                          20100929
view_count                                                              20695
webpage_url                       https://www.youtube.com/watch?v=Jc2F3-nawnI
transcript                  So I want to tell you a story -- an 

In [29]:
[row["title"]] + row["categories"]

NameError: name 'row' is not defined

In [30]:
talk_corpus = []
for (index,row) in df.iterrows():
    content = gensim.utils.simple_preprocess (row["transcript"])
    #tags = [row["title"], row["id"]] + row["categories"]'
    #tags = [row["title"]] + row["categories"]
    tags = [row["title"]] 
    tg = gensim.models.doc2vec.TaggedDocument(content,tags)
    talk_corpus.append(tg)

In [31]:
talk_corpus[0]

TaggedDocument(words=['so', 'want', 'to', 'tell', 'you', 'story', 'an', 'encouraging', 'story', 'about', 'addressing', 'desperation', 'depression', 'and', 'despair', 'in', 'afghanistan', 'and', 'what', 'we', 'have', 'learned', 'from', 'it', 'and', 'how', 'to', 'help', 'people', 'to', 'overcome', 'traumatic', 'experiences', 'and', 'how', 'to', 'help', 'them', 'to', 'regain', 'some', 'confidence', 'in', 'the', 'time', 'ahead', 'in', 'the', 'future', 'and', 'how', 'to', 'participate', 'again', 'in', 'everyday', 'life', 'so', 'am', 'jungian', 'psychoanalyst', 'and', 'went', 'to', 'afghanistan', 'in', 'january', 'by', 'chance', 'on', 'an', 'assignment', 'for', 'medica', 'mondiale', 'jung', 'in', 'afghanistan', 'you', 'get', 'the', 'picture', 'afghanistan', 'is', 'one', 'of', 'the', 'poorest', 'countries', 'in', 'the', 'world', 'and', 'percent', 'of', 'the', 'people', 'are', 'illiterate', 'war', 'and', 'malnutrition', 'kills', 'people', 'together', 'with', 'hope', 'you', 'may', 'know', 'this

In [ ]:
model = gensim.models.Doc2Vec(size = 300, 
                              min_count = 3, 
                              iter = 100)

In [ ]:
model.build_vocab(talk_corpus)
print("model's vocabulary length:", len(model.wv.vocab))

In [ ]:
model.train(talk_corpus,total_examples=model.corpus_count,epochs=model.epochs)

In [ ]:
model.docvecs.most_similar('My escape from North Korea | Hyeonseo Lee')

In [ ]:
df.head(1)

In [ ]:
content = gensim.utils.simple_preprocess (df.iloc[0]["transcript"])

In [ ]:
content

In [ ]:
new_vector = model.infer_vector(content)
sims = model.docvecs.most_similar([new_vector],topn=20)
sims